# YouTube Trending Videos Statistical Analysis

## INTRODUCTION

#### to be edited</b>
<b>https://www.kaggle.com/rsrishav/youtube-trending-video-dataset

In [1]:
# Importing libraries
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
from scipy import stats
from scipy.stats import norm
from scipy.stats import t
from scipy.stats import ttest_1samp
from sklearn_metrics import auc

%matplotlib inline

ModuleNotFoundError: No module named 'sklearn_metrics'

In [3]:
'''
Later on we can also remove the ones we wouldn't use
'''
# Importing dataset
data_gb = pd.read_csv("../data/GB_youtube_trending_data.csv",sep=",")  #uk  
data_fr = pd.read_csv("../data/FR_youtube_trending_data.csv",sep=",")  #france
data_us = pd.read_csv("../data/US_youtube_trending_data.csv",sep=",")  #usa 
data_ru = pd.read_csv("../data/RU_youtube_trending_data.csv",sep=",")  #russia
data_de = pd.read_csv("../data/DE_youtube_trending_data.csv",sep=",")  #germany 
data_ca = pd.read_csv("../data/CA_youtube_trending_data.csv",sep=",")  #canada   
data_kr = pd.read_csv("../data/KR_youtube_trending_data.csv",sep=",")  #southkorea   
data_jp = pd.read_csv("../data/JP_youtube_trending_data.csv",sep=",")  #japan
data_br = pd.read_csv("../data/BR_youtube_trending_data.csv",sep=",")  #brazil
data_mx = pd.read_csv("../data/MX_youtube_trending_data.csv",sep=",")  #mexico

In [4]:
'''
This can stay for now to easily see what are our properties in the dataset. We can delete it once
we are familiar with the dataset
'''
brief_list_col = []
brief_list = []

for i in range(0,len(data_gb.columns)):
    brief_list_col.append(data_gb.columns[i])
    brief_list.append(data_gb[data_gb.columns[i]][0])

brief_df = pd.DataFrame(brief_list_col,columns=["Property"])
brief_df.insert(1,"Output",brief_list,True)
brief_df

,Property,Output
0,video_id,J78aPJ3VyNs
1,title,I left youtube for a month and THIS is what ha...
2,publishedAt,2020-08-11T16:34:06Z
3,channelId,UCYzPXprvl5Y-Sf0g4vX-m6g
4,channelTitle,jacksepticeye
5,categoryId,24
6,trending_date,2020-08-12T00:00:00Z
7,tags,jacksepticeye|funny|funny meme|memes|jacksepti...
8,view_count,2038853
9,likes,353790


In [6]:
'''
Adding json files
'''

gb_json = pd.read_json('../data/GB_category_id.json')
fr_json = pd.read_json('../data/FR_category_id.json')
us_json = pd.read_json('../data/US_category_id.json')
ru_json = pd.read_json('../data/RU_category_id.json')
de_json = pd.read_json('../data/DE_category_id.json')
ca_json = pd.read_json('../data/CA_category_id.json')
kr_json = pd.read_json('../data/KR_category_id.json')
jp_json = pd.read_json('../data/JP_category_id.json')
br_json = pd.read_json('../data/BR_category_id.json')
mx_json = pd.read_json('../data/MX_category_id.json')

In [7]:
'''
Defining function to flat json files to get rid of dictionary format for every country
'''

col_list = ["items","items_snippet"]

def flat_func(df,column):
    for column in col_list:
        flatten = pd.DataFrame(dict(df[column])).transpose()
        columns = [str(i) for i in flatten.columns]
        flatten.columns = [column + "_" + str(colname) for colname in columns]
        df = pd.concat([df,flatten],axis=1)
        df = df.drop(column,axis=1)
    return df

In [8]:
'''
Creating flat databases
'''

gb_flat = flat_func(gb_json,col_list)[["items_id","items_snippet_title"]]
fr_flat = flat_func(fr_json,col_list)[["items_id","items_snippet_title"]]
us_flat = flat_func(us_json,col_list)[["items_id","items_snippet_title"]]
ru_flat = flat_func(ru_json,col_list)[["items_id","items_snippet_title"]]
de_flat = flat_func(de_json,col_list)[["items_id","items_snippet_title"]]
ca_flat = flat_func(ca_json,col_list)[["items_id","items_snippet_title"]]
kr_flat = flat_func(kr_json,col_list)[["items_id","items_snippet_title"]]
jp_flat = flat_func(jp_json,col_list)[["items_id","items_snippet_title"]]
br_flat = flat_func(br_json,col_list)[["items_id","items_snippet_title"]]
mx_flat = flat_func(mx_json,col_list)[["items_id","items_snippet_title"]]

In [9]:
'''
categoryId 29 is missing in 7 JSON files so I am going to remove rows with categoryId 29 in order
to run below code. There are only 3 rows available with categoryId 29 (for UK), so shouldn't be a 
problem.


Dropping categoryId 29 for each country
'''
drop_idx = data_gb[data_gb.categoryId == 29].index
data_gb = data_gb.drop(drop_idx)

drop_idx = data_fr[data_fr.categoryId == 29].index
data_fr = data_fr.drop(drop_idx)

drop_idx = data_us[data_us.categoryId == 29].index
data_us = data_us.drop(drop_idx)

drop_idx = data_ru[data_ru.categoryId == 29].index
data_ru = data_ru.drop(drop_idx)

drop_idx = data_de[data_de.categoryId == 29].index
data_de = data_de.drop(drop_idx)

drop_idx = data_ca[data_ca.categoryId == 29].index
data_ca = data_ca.drop(drop_idx)

drop_idx = data_kr[data_kr.categoryId == 29].index
data_kr = data_kr.drop(drop_idx)

drop_idx = data_jp[data_jp.categoryId == 29].index
data_jp = data_jp.drop(drop_idx)

drop_idx = data_br[data_br.categoryId == 29].index
data_br = data_br.drop(drop_idx)

drop_idx = data_mx[data_mx.categoryId == 29].index
data_mx = data_mx.drop(drop_idx)

In [10]:
'''
Defining function to add trend name next to categoryId for every country
'''
def trend_adder(country_df,country_flat):
    trend = []
    for trend_id in country_df["categoryId"]:
        for i in range(0,len(country_flat)):
            if trend_id == int(country_flat["items_id"][i]):
                trend.append(country_flat["items_snippet_title"][i])

    country_df.insert(6,"trend",trend)     

In [11]:
'''
Adding trend column next to categoryId for every country
'''
trend_order_data_country = [data_gb,data_fr,data_us,data_ru,data_de,
                            data_ca,data_kr,data_jp,data_br,data_mx]

trend_order_data_flat = [gb_flat,fr_flat,us_flat,ru_flat,de_flat,
                         ca_flat,kr_flat,jp_flat,br_flat,mx_flat]

for i in range(0,len(trend_order_data_country)):
    trend_adder(trend_order_data_country[i],trend_order_data_flat[i])

In [24]:
# Scrapping the most updated row for each link
latest_idx = []
for link in np.array(data_gb["video_id"].unique()):
    latest_row = data_gb[(data_gb["video_id"] == 
                          link)][["view_count"]].nlargest(1,"view_count").index[0]
    latest_idx.append(latest_row)

# Re-creating database based on unique video_id's (the most recent column is taken)
data_gb_unique = data_gb[data_gb.index.isin(latest_idx)]
data_us_unique = data_us[data_us.index.isin(latest_idx)]

In [25]:
data_gb_unique.columns

Index(['video_id', 'title', 'publishedAt', 'channelId', 'channelTitle',
       'categoryId', 'trend', 'trending_date', 'tags', 'view_count', 'likes',
       'dislikes', 'comment_count', 'thumbnail_link', 'comments_disabled',
       'ratings_disabled', 'description'],
      dtype='object')

## Statistical Analysis

In [26]:
data_gb_unique['view_count'].mean()

2128238.6995253745

In [28]:
data_gb_unique['view_count'].std()

7214438.945745977

In [27]:
data_us_unique['view_count'].mean()

3041465.1761052245

In [29]:
data_us_unique['view_count'].std()

9549893.37796264

Possible hypothesis;

more popular the video more like/less dislike,
tbc